# Real-Time Intelligence: KQL Queries for Casino Monitoring

**Notebook:** `01_eventhouse_kql_queries`  
**Component:** Eventhouse / KQL Database  
**Purpose:** Real-time casino floor monitoring queries

---

## Overview

This notebook contains KQL queries for real-time monitoring of casino operations using Fabric Eventhouse.

## Connection Setup

In [ ]:
# Eventhouse connection parameters
EVENTHOUSE_URI = "https://your-eventhouse.kusto.fabric.microsoft.com"
DATABASE_NAME = "CasinoRealTime"

# In Fabric, KQL queries can be run directly in KQL Querysets
# This notebook documents the queries for reference

## Real-Time Floor Status

```kql
// Real-time floor status - machines active in last 5 minutes
SlotTelemetry
| where ingestion_time() > ago(5m)
| summarize 
    LastActivity = max(EventTimestamp),
    TotalSpins = countif(EventType == "SPIN"),
    CoinIn = sum(BetAmount),
    CoinOut = sum(WinAmount)
  by MachineId, FloorLocation, CasinoId
| extend 
    NetRevenue = CoinIn - CoinOut,
    Status = iff(LastActivity > ago(1m), "Active", "Idle")
| order by NetRevenue desc
```

## Jackpot Alerts

```kql
// Recent jackpots requiring W-2G reporting
SlotTelemetry
| where EventType == "JACKPOT" or WinAmount >= 1200
| where ingestion_time() > ago(1h)
| project 
    EventTimestamp,
    MachineId,
    FloorLocation,
    PlayerId,
    WinAmount,
    RequiresW2G = WinAmount >= 1200,
    RequiresHandpay = WinAmount >= 1200
| order by EventTimestamp desc
```

## Machine Performance Anomalies

```kql
// Detect machines with unusual hold percentage
let expectedHold = 8.0; // 8% theoretical hold
let threshold = 3.0;    // Alert if +/- 3%
SlotTelemetry
| where EventType == "SPIN"
| where ingestion_time() > ago(1h)
| summarize 
    CoinIn = sum(BetAmount),
    CoinOut = sum(WinAmount),
    SpinCount = count()
  by MachineId, FloorLocation
| where SpinCount > 100  // Minimum sample size
| extend 
    ActualHold = (CoinIn - CoinOut) / CoinIn * 100,
    Deviation = abs(((CoinIn - CoinOut) / CoinIn * 100) - expectedHold)
| where Deviation > threshold
| extend 
    AlertType = iff(ActualHold > expectedHold + threshold, "High Hold", "Low Hold"),
    Severity = case(
        Deviation > 10, "Critical",
        Deviation > 5, "Warning",
        "Info")
| order by Deviation desc
```

## Revenue Trending (15-min windows)

```kql
// Revenue trend by 15-minute buckets
SlotTelemetry
| where EventType == "SPIN"
| where ingestion_time() > ago(4h)
| summarize 
    CoinIn = sum(BetAmount),
    CoinOut = sum(WinAmount),
    Transactions = count(),
    UniquePlayers = dcount(PlayerId)
  by bin(EventTimestamp, 15m), CasinoId
| extend NetRevenue = CoinIn - CoinOut
| order by EventTimestamp asc
| render timechart
```

## Security Alerts - Door Opens

```kql
// Machine door open events requiring attention
SlotTelemetry
| where EventType in ("DOOR_OPEN", "DOOR_CLOSE")
| where ingestion_time() > ago(30m)
| summarize 
    Events = make_list(pack("time", EventTimestamp, "type", EventType)),
    DoorOpens = countif(EventType == "DOOR_OPEN"),
    DoorCloses = countif(EventType == "DOOR_CLOSE")
  by MachineId, FloorLocation
| extend 
    IsOpen = DoorOpens > DoorCloses,
    AlertRequired = DoorOpens > DoorCloses
| where AlertRequired == true
```

## Player Session Tracking

```kql
// Active player sessions with real-time stats
SlotTelemetry
| where ingestion_time() > ago(30m)
| where isnotempty(PlayerId) and PlayerId != "ANONYMOUS"
| summarize 
    SessionStart = min(EventTimestamp),
    LastActivity = max(EventTimestamp),
    TotalBets = sum(BetAmount),
    TotalWins = sum(WinAmount),
    SpinCount = countif(EventType == "SPIN"),
    MachinesPlayed = dcount(MachineId)
  by PlayerId, SessionId
| extend 
    SessionDuration = datetime_diff('minute', LastActivity, SessionStart),
    NetResult = TotalWins - TotalBets,
    IsWinning = TotalWins > TotalBets,
    AvgBetSize = TotalBets / SpinCount
| where LastActivity > ago(5m)  // Currently active
| order by TotalBets desc
```

## Compliance Monitoring - CTR Threshold

```kql
// Players approaching CTR threshold ($10,000)
SlotTelemetry
| where ingestion_time() > ago(24h)
| where isnotempty(PlayerId) and PlayerId != "ANONYMOUS"
| summarize 
    TotalCashIn = sumif(BetAmount, EventType == "CASH_IN"),
    TotalCashOut = sumif(WinAmount, EventType == "CASH_OUT")
  by PlayerId
| extend TotalCashActivity = TotalCashIn + TotalCashOut
| where TotalCashActivity >= 8000  // Alert at 80% of threshold
| extend 
    PercentOfThreshold = TotalCashActivity / 10000 * 100,
    RequiresCTR = TotalCashActivity >= 10000,
    AlertLevel = case(
        TotalCashActivity >= 10000, "CTR Required",
        TotalCashActivity >= 9000, "Critical",
        "Warning")
| order by TotalCashActivity desc
```

## Real-Time Dashboard Queries

These queries power the real-time dashboard tiles:

### Active Machines Count
```kql
SlotTelemetry
| where ingestion_time() > ago(5m)
| summarize ActiveMachines = dcount(MachineId)
```

### Current Hour Revenue
```kql
SlotTelemetry
| where EventType == "SPIN"
| where EventTimestamp > startofhour(now())
| summarize 
    Revenue = sum(BetAmount) - sum(WinAmount)
```

### Active Players
```kql
SlotTelemetry
| where ingestion_time() > ago(5m)
| where isnotempty(PlayerId) and PlayerId != "ANONYMOUS"
| summarize ActivePlayers = dcount(PlayerId)
```